# Gaussian Noise Simulation Environment

1. Load a clean signal acquired using GNURadio.
1. Add different percentage of Gaussian Noise to the ideal signal to simulate the behaviour of a blocking card.
1. Analyze the performance of the demodulator.

In [ ]:
from common_imports import *


# Useful in Windwos OS
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# PATHS
DATA_PATH     = r'../data/classic/'
RESULTS_PATH  = r'../data/results/'

# TO SHOW PLOTS
SHOW_PLOTS = True

### **1.** Load the signal of a clean communication

In [ ]:
# Paths
dataPath = DATA_PATH + 'classic_clean.raw'
expected_file = libnfc_file = DATA_PATH + 'expectedTxt/classic_clean.txt'

# Load data
data = load_mag(dataPath)

# Print the signal
plt.figure(figsize=(30, 4), color="#47B5FF"
plt.plot(data)
plt.show()

In [ ]:
# Load everything into a NfcSignal class
start = time()
s = NfcSignal(data, 
      expected_file = expected_file, 
      libnfc_file = libnfc_file,
      attack_mode = 0, 
      mean_samples = 0, 
      message_batch_size = 8)
end = time()
print(f"init duration {end-start}")

# Start Demodulation
s.start_demodulation()

Check demodulation stats on a clean signal

In [ ]:
s.demodulation_stats()

### **2.** Add noise to the clean signal

#### Add gaussian noise at different percentage: 5%, 10%, 15%, 20%, 25%, 30%

In [ ]:
# Function to generate Gaussian Noise
def gaussian_noise(x,mu,std):
    noise = np.random.normal(mu, std, size = x.shape)
    return noise

In [ ]:
import matplotlib 
matplotlib.rcParams.update({'font.size': 16})

signals = []
mu = 0.0
percentageSTD = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

for p in percentageSTD:
    print("\n💻 [CREATE SIGNAL WITH {}% OF GAUSSIAN NOISE]\n".format(p*100))
    if p != 0.2:
        continue
    noise = gaussian_noise(data, mu, p * np.std(data))
    noisyData = data + noise
    
    if(SHOW_PLOTS):
        # Print the signal
        print("📡 [SIGNAL]:")
        plt.figure(figsize = (12.5, 4))
        plt.plot(noisyData, label ="Noisy signal", color="")
        plt.plot(data, color = "orange", label ="Original signal")
        plt.legend(fontsize=16)
        plt.xlabel('Time [s]')
        plt.ylabel('Voltage [V]')
        plt.tight_layout()
        plt.savefig("gaussian_noise_"+ str(p*100) +".pdf", bbox_inches = 'tight', pad_inches = 0)
        plt.show()

    signals.append(noisyData)

### **3.** Analyze the performance of the demodulator on noisy signals

In [ ]:
NOISE_TYPE_LIST = []
resultsDF = pd.DataFrame(columns =  ['noise_type','reader_correct', 'reader_expected', 'tag_detected', 'tag_correct', 'tag_expected', 'total_correct', 'total_expected'])

In [ ]:
for i in range(len(signals)):
    print("\n💻 [SIGNAL WITH {}% OF GAUSSIAN NOISE ]\n".format(percentageSTD[i]*100))

    start = time()
    s = NfcSignal(signals[i], 
        expected_file = expected_file, 
        libnfc_file = libnfc_file,
        attack_mode = 0, 
        mean_samples = 0, 
        message_batch_size = 8)
    end = time()
    print(f"init duration {end-start}")

    # Start Demodulation
    s.start_demodulation()

    # Get demodulation stats and add blockig card blockingCardName
    print("📝 STATS: ")
    tmpDF = s.demodulation_stats()
    tmpDF['noise_type'] = "gaussian_noise_{}".format(percentageSTD[i]*100)
    NOISE_TYPE_LIST.append("gaussian_{}".format(percentageSTD[i]*100))

    # Add demodulation stats to the results DF
    resultsDF = pd.concat([resultsDF, tmpDF], ignore_index=True)

In [ ]:
resultsDF

#### Compute Demodulation Rates

In [ ]:
# Lists to store rates
readerDemodulationRateList = []
tagDemodulationRateList = []
tagDetectionRateList = []
totalDemodulationRateList = []

# Iterate through rows
for index, row in resultsDF.iterrows():
    readerDemodulationRateList.append(round(row['reader_correct']/row['reader_expected'],2))
    tagDetectionRateList.append(round(row['tag_detected']/row['tag_expected'],2))
    tagDemodulationRateList.append(round(row['tag_correct']/row['tag_expected'],2))
    totalDemodulationRateList.append(round(row['total_correct']/row['total_expected'],2))

# Add rates to resultsDF
resultsDF['reader_demodulation_rate'] = readerDemodulationRateList
resultsDF['tag_detection_rate'] = tagDetectionRateList
resultsDF['tag_demodulation_rate'] = tagDemodulationRateList
resultsDF['total_demodulation_rate'] = totalDemodulationRateList

In [ ]:
resultsDF

### **4.** Save stats into CSV file

In [ ]:
# Save to CSV
resultsDF.to_csv(RESULTS_PATH + "gaussian_results.csv")  

In [ ]:
# Load from CSV
resultsDF = pd.read_csv(RESULTS_PATH + "gaussian_results.csv", index_col = 0) 
print(resultsDF)